## __ INIT __

In [1]:
from services.db.supabase_services import supabase_client
supabase = supabase_client()

from services.twilio import client

from app.core.config import settings
import requests
import json
from typing import Dict, Annotated, Optional, List, Union
import os 
from dotenv import load_dotenv
load_dotenv()

from services.cache import get_agent_metadata, get_all_agents

from services.chat.chat import similarity_search, get_embedding

In [16]:
from services.cache import get_all_agents, call_data, get_agent_metadata, initialize_calendar_cache

async def get_agent_id(room_name: str):
    agents = await get_all_agents()
    def get_agent_id_by_phone(data, phone_number):
        print(f"data: {data}")
        print(f"phone_number: {phone_number}")
        for agent in data:
            print(f"agent: {agent}")
            if agent.get('assigned_telephone') == phone_number:
                return agent.get('id')
        return None
    
    # import re
    # pattern = r'(?:\+)?(\d{12})'
    # match = re.search(pattern, room_name)
    # if match:
    #     caller_number = match.group(1)
    print("\n\n\n\n\nroom_name:", room_name)
    print("call_data:", call_data)
    twilio_number = call_data.get(room_name)
    print(f"twilio_number: {twilio_number}")
    agent_id = get_agent_id_by_phone(agents, twilio_number)
    print(f"agent_id: {agent_id}")
    return agent_id

In [17]:

yalla = await get_agent_id("call-_+447745187688_wbEVj3gbVK6J")
print(f"yalla: {yalla}")






room_name: call-_+447745187688_wbEVj3gbVK6J
call_data: {}
twilio_number: None
data: [{'id': '1bf662cf-4d01-4c82-b919-8534ad071380', 'agentName': 'Liquidate My Business', 'userId': 'user_2mmXezcGmjZCf88gT2v2waCBsXv', 'agentPurpose': 'question-answer', 'instructions': 'You are a receptionist for an insolvency practitioner firm called "Liquidate My Business". Your job is to qualify and answer questions that a business has.', 'dataSource': '[{"id": "1", "title": "http://flowon.ai", "data_type": "web"}, {"id": 2, "title": "https://developer.nylas.com", "data_type": "web"}]', 'openingLine': 'Liquidate my business, how can I help? ', 'voice': '95856005-0332-41b0-935f-352e296aa0df', 'language': 'en-GB', 'features': {}, 'assigned_telephone': None, 'datetime_creation': '2024-11-09T14:30:04.666584+00:00', 'form_fields': {'fields': [{'type': 'text', 'label': 'Full Name'}, {'type': 'email', 'label': 'Email Address'}]}, 'voiceProvider': 'cartesia', 'notify': None}, {'id': 'a187b3ab-7c2f-44aa-b7

: 

In [ ]:
def get_twilio_countries() -> List[str]:
    countries = client.available_phone_numbers.list()
    return [country.country_code for country in countries]

get_twilio_countries()

In [44]:
def get_available_number_types(client, country_code):
    number_types = ['local', 'toll_free', 'mobile', 'national']
    available_numbers = {}
    
    for number_type in number_types:
        try:
            # Try to list up to 5 numbers of each type
            numbers = getattr(client.available_phone_numbers(country_code), number_type).list(limit=5)
            numbers_list = [number.phone_number for number in numbers]
            
            # Only add to dictionary if numbers were found
            if numbers_list:
                available_numbers[number_type] = numbers_list
                
        except Exception as e:
            continue
            
    return available_numbers

country_code = "EE"
available_types = get_available_number_types(client, country_code)
print(f"Available number types for {country_code}: {available_types}")

country_code = "GB"
us_types = get_available_number_types(client, country_code)
print(f"Available number types for {country_code}: {us_types}")

Available number types for EE: {'local': ['+3726692342', '+3726361070', '+3726360845', '+3726692304', '+3726360819'], 'toll_free': ['+37280010004', '+37280010009', '+37280010003', '+37280010006', '+37280010008'], 'mobile': ['+37256011399', '+37257052018', '+37257052005', '+37257052016', '+37256011380']}
Available number types for GB: {'local': ['+441848260780', '+441379778515', '+441544302914', '+442045722228', '+441325808634'], 'toll_free': ['+448085012991', '+448085012755', '+448085012630', '+448085020332', '+448085012962'], 'mobile': ['+447897014832', '+447458694422', '+447401608313', '+447575732921', '+447883318090']}


## In_mem kb

In [35]:
import faiss
import numpy as np

# Initialize an in-memory index
dimension = 1024  
index = faiss.IndexFlatL2(dimension)

# Add vectors
vectors = np.random.random((1000, dimension)).astype('float32')
index.add(vectors)

# Perform similarity search
query = np.random.random((1, dimension)).astype('float32')
k = 5  # Number of nearest neighbors
distances, indices = index.search(query, k)

In [40]:
async def get_user_kb(user_id: str) :
    try:
        # Fetch chunks for the user
        chunks_response = supabase.table('chunks') \
            .select('*') \
            .eq('user_id', user_id) \
            .execute()

        # Fetch web data for the user
        web_data_response = supabase.table('user_web_data') \
            .select('*') \
            .eq('user_id', user_id) \
            .execute()

        return {
            'chunks': chunks_response.data,
            'web_data': web_data_response.data
        }
    except Exception as e:
        print(f"Error fetching user data: {str(e)}")
        return None
    
user_kb: Union[Dict, None] = await get_user_kb("user_2mmXezcGmjZCf88gT2v2waCBsXv")

async def filter_agent_kb(data: Union[Dict, None], 
                        data_source: Dict[str, List[Union[str, int]]]):
    data_source: Dict = json.loads(data_source)
    data_source: Dict = {
        "web": [item['title'] for item in data_source if item['data_type'] == 'web'],
        "text_files": [item['id'] for item in data_source if item['data_type'] != 'web']
    }     

    if not data:
        return {'web_data': [], 'chunks': []}
    
    return {
        'web_data': [item for item in data.get('web_data', []) 
                    if item.get('root_url') in data_source.get('web', [])],
        'chunks': [item for item in data.get('chunks', []) 
                if item.get('parent_id') in data_source.get('text_files', [])]}

async def similarity_search_db(data_source: str, query: str):
    if data_source != "all":
        data_source: Dict = json.loads(data_source)
        data_source: Dict = {
            "web": [item['title'] for item in data_source if item['data_type'] == 'web'],
            "text_files": [item['id'] for item in data_source if item['data_type'] != 'web']
        }
        results = await similarity_search(query, data_source=data_source, user_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")
        
        print("supabase results:", results)
    elif data_source == "all":
        data_source = {"web": ["all"], "text_files": ["all"]}
        results = await similarity_search(query, data_source=data_source, user_id="user_2mmXezcGmjZCf88gT2v2waCBsXv")
        
        print("supabase results:", results)
        return results
    

query = "what is the cost of an oil change"


In [ ]:
agent_metadata: Dict = await get_agent_metadata("aaf5fce2-c925-4a32-aefc-e4af35d4b8e1")
data_source: str = agent_metadata.get('dataSource', None)

agent_kb_data = await filter_agent_kb(data=user_kb, data_source=data_source)

agent_kb_data


## LLM TEXT-TEXT

In [ ]:
from livekit.agents import llm
from livekit.plugins import openai  # or anthropic

async def chat_with_llm():
    # Initialize the LLM
    llm_instance = openai.LLM()  # or anthropic.LLM()
    
    # Create initial chat context
    chat_ctx = llm.ChatContext()
    
    # Add system message if desired
    chat_ctx.append(
        role="system",
        text="You are a helpful assistant."
    )
    
    # Add user message
    chat_ctx.append(
        role="user", 
        text="Hello, how are you?"
    )
    
    # Get response stream
    response_stream = llm_instance.chat(chat_ctx=chat_ctx)
    
    # Collect the response
    full_response = ""
    async for chunk in response_stream:
        # Each chunk.choices[0].delta.content contains a text fragment
        if chunk.choices[0].delta.content:
            text_fragment = chunk.choices[0].delta.content
            full_response += text_fragment
            print(text_fragment, end="", flush=True)  # For real-time output
    
    #print("\nFull response:", full_response)

await chat_with_llm()
# Run with:
# asyncio.run(chat_with_llm())

In [ ]:
from livekit.agents import llm
from livekit.plugins import openai
from livekit.agents.llm import USE_DOCSTRING
from services.cache import get_agent_metadata

from services.chat.chat import similarity_search
import logging
logger = logging.getLogger(__name__)

@llm.ai_callable(
    name="question_and_answer",
    description="Extract user's question and perform information retrieval search to provide relevant answers",
    auto_retry=True
)
async def question_and_answer(
    question: Annotated[
        str,
        llm.TypeInfo(
            description="The user's question that needs to be answered"
        )
    ]
) -> str:
    """
    Performs information retrieval search based on the user's question.
    Returns relevant information found in the knowledge base.
    """
    try:
        logger.info(f"Processing Q&A for question: {question}")
        room_name = "agent_f33b321f-cd04-4db4-b433-57801ac588d4_room_visitor_02c0f9d7-0343-4a18-9800-d74ae75df057"

        agent_id = room_name.split('_')[1]  # Extract agent_id from room name
        agent_metadata: Dict = await get_agent_metadata(agent_id)

        user_id: str = agent_metadata['userId']
        data_source: str = agent_metadata.get('dataSource', None)
        
        if data_source != "all":
            data_source: Dict = json.loads(data_source)
            data_source: Dict = {
                "web": [item['title'] for item in data_source if item['data_type'] == 'web'],
                "text_files": [item['id'] for item in data_source if item['data_type'] != 'web']
            }

            print("data_source:", data_source)
            results = await similarity_search(question, data_source=data_source, user_id=user_id)
            print("results:", results)
        else:
            data_source = {"web": ["all"], "text_files": ["all"]}
            results = await similarity_search(question, data_source=data_source, user_id=user_id)

        return f"Based on your question, here's what I found:\n{results}"
        
    except Exception as e:
        logger.error(f"Error in question_and_answer: {str(e)}", exc_info=True)
        return "I apologize, but I encountered an error while searching for an answer to your question."

async def get_llm_response(chat_ctx: llm.ChatContext, fnc_ctx: llm.FunctionContext = None):
    print("\n\n\n\n GET LLM RESPONSE")
    # Get response stream with function context
    if fnc_ctx: 
        llm_instance = openai.LLM()
        response_stream = llm_instance.chat(
            chat_ctx=chat_ctx,
            fnc_ctx=fnc_ctx
                    )
    else:
        llm_instance = openai.LLM()
        response_stream = llm_instance.chat(
            chat_ctx=chat_ctx
        )


    async for chunk in response_stream:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)
        elif chunk.choices[0].delta.tool_calls:
            # Handle function calls
            for tool_call in chunk.choices[0].delta.tool_calls:
                print(f"\nFunction called: {tool_call.function_info.name}")
                print(f"Arguments: {tool_call.arguments}")
                
                # Execute the function
                called_function = tool_call.execute()
                result = await called_function.task
                print(f"Function result: {result}")

async def test_llm_function_calls():
    
    # Create function context
    fnc_ctx = llm.FunctionContext()
    fnc_ctx._register_ai_function(question_and_answer)
    
    # Create chat context
    chat_ctx = llm.ChatContext()
    chat_ctx.append(
        role="user",
        text="tell me about headformers"
    )
    
    await get_llm_response(chat_ctx, fnc_ctx)

await test_llm_function_calls()
# Run with:
# asyncio.run(test_llm_function_calls())

## Anthropic pdf 

In [ ]:
import anthropic
import base64
import httpx
import os 
from dotenv import load_dotenv

load_dotenv()

# First fetch the file
pdf_url = "https://assets.anthropic.com/m/1cd9d098ac3e6467/original/Claude-3-Model-Card-October-Addendum.pdf"
pdf_data = base64.standard_b64encode(httpx.get(pdf_url).content).decode("utf-8")


# Finally send the API request
client = anthropic.Anthropic()
message = client.beta.messages.create(
    model="claude-3-5-sonnet-20241022",
    betas=["pdfs-2024-09-25"],
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "document",
                    "source": {
                        "type": "base64",
                        "media_type": "application/pdf",
                        "data": pdf_data
                    }
                },
                {
                    "type": "text",
                    "text": "Which model has the highest human preference win rates across each use-case?"
                }
            ]
        }
    ],
)

print(message)


## DataSource

In [ ]:
from services.chat.chat import similarity_search

await similarity_search("pre packaged admin", "all")

In [6]:
agents_metadata = await get_agent_metadata("ae1d8dc1-ef3a-4b94-98d3-d78f0a4ad494")
agents_metadata['features']['features']

['qa', 'lead_gen']

In [12]:
from services.cache import get_agent_metadata
import json 
from services.db.supabase_services import supabase_client
supabase = supabase_client()

agents_metadata = await get_agent_metadata("13400af9-0655-46bc-a815-9664910c2abc")
data_source = agents_metadata.get('dataSource', None)
if data_source != "all":
    data_source = json.loads(data_source)
    data_source = {
                "web": [item['title'] for item in data_source if item['data_type'] == 'web'],
                "text_files": [item['id'] for item in data_source if item['data_type'] != 'web']
            }

elif data_source == "all":
    data_source



In [7]:
from services.chat.chat import get_embedding
query_embedding = await get_embedding("jina embedding model")

## NYLAS API

In [32]:
import os 
from dotenv import load_dotenv
from nylas import Client
from typing import List, Dict, Any
import requests

load_dotenv()

NYLAS_API_KEY = os.getenv("NYLAS_API_KEY")
NYLAS_API_URI = os.getenv("NYLAS_API_URI")

# Initialize Nylas client
nylas = Client(
    api_key = NYLAS_API_KEY,
    api_uri = NYLAS_API_URI
)

In [ ]:
""" NYLAS WEBHOOK SET UP """

from dotenv import load_dotenv
load_dotenv()

import os
import sys
from nylas import Client
from nylas.models.webhooks import WebhookTriggers

nylas = Client(
  os.environ.get('NYLAS_API_KEY'),
  os.environ.get('NYLAS_API_URI')
)

grant_id = "5ef0555c-25ab-4b4e-b4a1-02fd8ba4d255"
webhook_url = "https://internally-wise-spaniel.eu.ngrok.io/api/v1/nylas/webhook"

email = os.environ.get("EMAIL")

webhook = nylas.webhooks.create(
  request_body={
    "trigger_types": [WebhookTriggers.EVENT_CREATED],
    "webhook_url": webhook_url,
    "description": "My first webhook",
    "notification_email_address": email,
  }
)

print(webhook)   

In [ ]:
""" WORKING SEND EMAIL """

grant_id = "5ef0555c-25ab-4b4e-b4a1-02fd8ba4d255"
email = "michael@flowon.ai"

#attachment = utils.file_utils.attach_file_request_builder("Nylas_Logo.png")

message = nylas.messages.send(
  grant_id,
  request_body={
    "to": [{ "name": "Name", "email": email }],
    "reply_to": [{ "name": "Name", "email": email }],
    "subject": "Your Subject Here",
    "body": "Your email body here.",
  }
)

print(message)   

In [5]:
""" get calendar id"""
grant_id = "5ef0555c-25ab-4b4e-b4a1-02fd8ba4d255"

calendar = nylas.calendars.find(
    grant_id,
    "primary"
)

print(calendar)   

(Calendar(id='AAkALgAAAAAAHYQDEapmEc2byACqAC-EWg0AT6mu_rvDikK57fYroNKSNAAEWMJ6ZAAA', grant_id='5ef0555c-25ab-4b4e-b4a1-02fd8ba4d255', name='Calendar', read_only=False, is_owned_by_user=True, object='calendar', timezone=None, description=None, location=None, hex_color='#cf2b36', hex_foreground_color=None, is_primary=True, metadata=None), '3879748553-464e1a02-7382-462f-a3d6-7302acbdae06')


In [30]:
""" GET CALENDAR EVENTS """
calendar_id = "AAkALgAAAAAAHYQDEapmEc2byACqAC-EWg0AT6mu_rvDikK57fYroNKSNAAEWMJ6ZAAA"

def get_calendar_events(grant_id: str, calendar_id: str = "primary", limit: int = 100) -> List[Dict[Any, Any]]:
    """
    Fetch calendar events for a given grant (email) from Nylas API.
    """
    # Use the v3 endpoint format
    url = f"{NYLAS_API_URI}/v3/grants/{grant_id}/events"
    
    headers = {
        "Accept": "application/json",
        "Authorization": f"Bearer {NYLAS_API_KEY}",
        "Content-Type": "application/json"
    }
    
    params = {
        "calendar_id": calendar_id,
        "limit": limit
    }
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    
    return response.json()

# Test the function
response = get_calendar_events("5ef0555c-25ab-4b4e-b4a1-02fd8ba4d255", calendar_id=calendar_id)


from datetime import datetime

start = response['data'][0]['when']['start_time']
end = response['data'][0]['when']['end_time']
start = datetime.fromtimestamp(start)
end = datetime.fromtimestamp(end)



#### SCHEDULAR WORKFLOW

In [43]:
""" CREATE SCHEDULING CONFIGURATION """

import requests
import json

url = "https://api.us.nylas.com/v3/grants/5ef0555c-25ab-4b4e-b4a1-02fd8ba4d255/scheduling/configurations"
headers = {
    "Accept": "application/json, application/gzip",
    "Authorization": f"Bearer {NYLAS_API_KEY}",
    "Content-Type": "application/json"
}

payload = {
    "requires_session_auth": False,
    "participants": [
        {
            "name": "Test",
            "email": "azkar9@hotmail.co.uk",
            "is_organizer": True,
            "availability": {
                "calendar_ids": ["primary"]
            },
            "booking": {
                "calendar_id": "primary"
            }
        }
    ],
    "availability": {
        "duration_minutes": 30
    },
    "event_booking": {
        "title": "My test event"
    }
}

response = requests.post(url, headers=headers, json=payload)
request_id = response.json()['data']['id']


{'request_id': '6125823212-2bd5fe22-48a2-457e-8212-a33b18466ff1', 'data': {'id': 'd3bc49b4-edd2-435d-a5f6-a8e608eeb787', 'slug': 'ddf9bd6a-7736-4b95-864a-e128a77156cd', 'participants': [{'email': 'azkar9@hotmail.co.uk', 'is_organizer': True, 'name': 'Test', 'availability': {'calendar_ids': ['primary']}, 'booking': {'calendar_id': 'primary'}}], 'requires_session_auth': False, 'availability': {'duration_minutes': 30, 'interval_minutes': 30}, 'event_booking': {'title': 'My test event', 'booking_type': 'booking', 'hide_participants': None, 'disable_emails': None}, 'scheduler': {'available_days_in_future': 30, 'min_cancellation_notice': 0, 'min_booking_notice': 60, 'email_template': {'booking_confirmed': {}}}, 'appearance': None}}


In [42]:
""" CREATE A SESSION """
""" config does not require a session for now.. """
import requests

url = "https://api.us.nylas.com/v3/scheduling/sessions"
headers = {
    "Accept": "application/json, application/gzip",
    "Authorization": f"Bearer {NYLAS_API_KEY}",
    "Content-Type": "application/json"
}
payload = {
    "configuration_id": f"{request_id}",
    "time_to_live": 10
}

response = requests.post(url, headers=headers, json=payload)

# Check if the request was successful
if response.status_code == 200:
    print("Success:", response.json())
else:
    print("Error:", response.status_code)
    print(response.text)

Error: 400
{"request_id":"7248723175-15b40b85-6589-4eff-8750-8e7f6f69e377","error":{"type":"invalid_request_error","message":"Configuration does not require a session"}}


In [76]:
# Given Unix timestamp
current_unix = 1731525206

# One week has 7 days * 24 hours * 60 minutes * 60 seconds = 604800 seconds
ONE_WEEK_IN_SECONDS = 7 * 24 * 60 * 60  # 604800

# Calculate one week ahead
one_week_ahead = current_unix + ONE_WEEK_IN_SECONDS

# Calculate two weeks ahead
two_weeks_ahead = current_unix + (2 * ONE_WEEK_IN_SECONDS)

# To verify the dates (optional)
from datetime import datetime
print(f"Current time: {datetime.fromtimestamp(current_unix)}")
print(f"One week ahead: {datetime.fromtimestamp(one_week_ahead)}")
print(f"Two weeks ahead: {datetime.fromtimestamp(two_weeks_ahead)}")

Current time: 2024-11-13 19:13:26
One week ahead: 2024-11-20 19:13:26
Two weeks ahead: 2024-11-27 19:13:26


In [73]:
""" GET SCHEDULING AVAILABILITY """

import requests
import time 

url = "https://api.us.nylas.com/v3/scheduling/availability"

current_unix = int(time.time())

# Parameters
params = {
    "start_time": current_unix,
    "end_time": current_unix + 1000000, #### NEED A WAY TO DEFINE THE END RANGE
    "configuration_id": f"{request_id}"  
}

headers = {
    "Accept": "application/json, application/gzip",
    "Content-Type": "application/json"
}

response = requests.get(url, params=params, headers=headers)

# Check if request was successful
if response.status_code == 200:
    availability_data = response.json()
    print(availability_data)
else:
    print(f"Error: {response.status_code}")
    print(response.text)

{'request_id': '4682746253-dfc73928-990d-46e5-b298-cf6e70627b80', 'data': {'time_slots': [{'emails': ['azkar9@hotmail.co.uk'], 'end_time': 1731527100, 'start_time': 1731525300}, {'emails': ['azkar9@hotmail.co.uk'], 'end_time': 1731528900, 'start_time': 1731527100}, {'emails': ['azkar9@hotmail.co.uk'], 'end_time': 1731530700, 'start_time': 1731528900}, {'emails': ['azkar9@hotmail.co.uk'], 'end_time': 1731532500, 'start_time': 1731530700}, {'emails': ['azkar9@hotmail.co.uk'], 'end_time': 1731534300, 'start_time': 1731532500}, {'emails': ['azkar9@hotmail.co.uk'], 'end_time': 1731536100, 'start_time': 1731534300}, {'emails': ['azkar9@hotmail.co.uk'], 'end_time': 1731537900, 'start_time': 1731536100}, {'emails': ['azkar9@hotmail.co.uk'], 'end_time': 1731539700, 'start_time': 1731537900}, {'emails': ['azkar9@hotmail.co.uk'], 'end_time': 1731541500, 'start_time': 1731539700}, {'emails': ['azkar9@hotmail.co.uk'], 'end_time': 1731543300, 'start_time': 1731541500}, {'emails': ['azkar9@hotmail.co

In [79]:
""" FORMAT TIME SLOTS TO LLM READABLE FORMAT"""
from datetime import datetime

def format_time_slots(time_slots):
    formatted_slots = []
    
    def get_time_of_day(hour):
        if 5 <= hour < 12:
            return "morning"
        elif 12 <= hour < 17:
            return "afternoon"
        elif 17 <= hour < 22:
            return "evening"
        else:
            return "night"
    
    for slot in time_slots:
        # Convert UNIX timestamps to datetime objects
        start_time = datetime.fromtimestamp(slot['start_time'])
        end_time = datetime.fromtimestamp(slot['end_time'])
        
        # Format the times in a human-readable way
        formatted_slot = {
            'date': start_time.strftime('%A, %B %d, %Y'),
            'time': f'{start_time.strftime("%I:%M %p")} - {end_time.strftime("%I:%M %p")}',
            'time_of_day': get_time_of_day(start_time.hour),
            # 'emails': slot['emails']
        }
        formatted_slots.append(formatted_slot)
    
    return formatted_slots

format_time_slots(response.json()['data']['time_slots'])


# """ HELPER: GET UNIX TIMESTAMP FOR A GIVEN DATE """
# # Create datetime object for 15th November 2024 at 23:59:59
# date = datetime(2024, 11, 15, 23, 59, 59)

# # Convert to Unix timestamp (seconds since epoch)
# unix_timestamp = int(date.timestamp())

# print(f"Unix timestamp for {date}: {unix_timestamp}")

[{'date': 'Wednesday, November 13, 2024',
  'time': '07:15 PM - 07:45 PM',
  'time_of_day': 'evening'},
 {'date': 'Wednesday, November 13, 2024',
  'time': '07:45 PM - 08:15 PM',
  'time_of_day': 'evening'},
 {'date': 'Wednesday, November 13, 2024',
  'time': '08:15 PM - 08:45 PM',
  'time_of_day': 'evening'},
 {'date': 'Wednesday, November 13, 2024',
  'time': '08:45 PM - 09:15 PM',
  'time_of_day': 'evening'},
 {'date': 'Wednesday, November 13, 2024',
  'time': '09:15 PM - 09:45 PM',
  'time_of_day': 'evening'},
 {'date': 'Wednesday, November 13, 2024',
  'time': '09:45 PM - 10:15 PM',
  'time_of_day': 'evening'},
 {'date': 'Wednesday, November 13, 2024',
  'time': '10:15 PM - 10:45 PM',
  'time_of_day': 'night'},
 {'date': 'Wednesday, November 13, 2024',
  'time': '10:45 PM - 11:15 PM',
  'time_of_day': 'night'},
 {'date': 'Wednesday, November 13, 2024',
  'time': '11:15 PM - 11:45 PM',
  'time_of_day': 'night'},
 {'date': 'Wednesday, November 13, 2024',
  'time': '11:45 PM - 12:15 

In [ ]:
from services.knowledge_base.kb import get_kb_items

response = await get_kb_items("user_2mmXezcGmjZCf88gT2v2waCBsXv")

In [ ]:
from itertools import groupby
from operator import itemgetter

def group_by_root_url(items):
    if not isinstance(items, list):
        raise TypeError(f"Expected a list, got {type(items)}")
    
    # Sort items by root_url
    sorted_items = sorted(items, key=itemgetter('root_url'))
    
    # Group items and create consolidated records
    result = []
    for root_url, group in groupby(sorted_items, key=itemgetter('root_url')):
        group_list = list(group)
        
        # Create consolidated record
        consolidated = {
            'title': root_url,  # Using root_url as title
            'root_url': root_url,
            'content': [{  # Group of URLs and their fields
                'url': item.get('url', ''),
                'id': item['id'],
                'token_count': item.get('token_count', 0)
            } for item in group_list],
            'created_at': next(iter(group_list)).get('created_at', ''),  # Take created_at from first item
            'data_type': 'web',
            'user_id': group_list[0].get('user_id')  # Assuming user_id is consistent within group
        }
        result.append(consolidated)
    
    return result

grouped = group_by_root_url(response)
grouped

In [ ]:
from services.chat.chat import similarity_search

await similarity_search("what is the cost of an oil change", table_names=["user_text_files"])